In [1]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.0.9000     ✔ purrr   0.3.4     
✔ tibble  3.0.1          ✔ dplyr   0.8.5     
✔ tidyr   1.0.3          ✔ stringr 1.4.0     
✔ readr   1.3.1          ✔ forcats 0.5.0     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



[1] "a6-essemble"

In [2]:
data <- read.csv("../input/a6-essemble/Earthquate_Damage.csv")
cat_cols <- c("land_surface_condition","foundation_type",
              "roof_type","ground_floor_type",
             "other_floor_type","position",
              "plan_configuration","legal_ownership_status")
install.packages("nnet")
library(nnet)
j <- 1
for(i in 1:length(cat_cols)){
    tmp <- class.ind(data[,cat_cols[i]])
    colnames(tmp) <- (j:(j+length(colnames(tmp)) - 1))
    data <- cbind(data,tmp)
    j <- (j+length(colnames(tmp)))
}
data <- data[,!(names(data)%in%cat_cols)]
scaled_data <- scale(data[,1:31],center=TRUE,scale=TRUE)
other_data <- data[,32:70]
data <- cbind(scaled_data,other_data)
rm(scaled_data)
rm(other_data)
set.seed(42)
trn_idx <- sample(1:nrow(data), 150000)
train <- data[trn_idx,]
tmp <- data[-trn_idx,]
val_idx <- sample(1:nrow(tmp),50000)
val <- tmp[val_idx,]
test <- tmp[-val_idx,]
rm(trn_idx)
rm(val_idx)
train <- rbind(train,val)
rm(val)
perf_eval_multi <- function(cm){
  
  # Simple Accuracy
  ACC = sum(diag(cm))/sum(cm)
  
  # Balanced Correction Rate
  BCR = 1
  for (i in 1:dim(cm)[1]){
    BCR = BCR*(cm[i,i]/sum(cm[i,])) 
  }
  
  BCR = BCR^(1/dim(cm)[1])
  
  return(c(ACC, BCR))
}
prediction_one <- function(pred){
    tmp <- apply(pred,1,which.max)
    for (i in 1:nrow(pred)){
        pred[i,tmp[i]] <- 1
        pred[i,-tmp[i]] <- 0
    }
    return(pred)
}
set.seed(42)
trn_idx <- sample(1:nrow(data), 150000)
train <- data[trn_idx,]
tmp <- data[-trn_idx,]
val_idx <- sample(1:nrow(tmp),50000)
val <- tmp[val_idx,]
test <- tmp[-val_idx,]
x_train <- train[,!(names(train)%in%"damage_grade")]
y_train <- train[,(names(train)%in%"damage_grade")]
y_train <- class.ind(y_train)
x_val <- val[,!(names(val)%in%"damage_grade")]
y_val <- val[,(names(val)%in%"damage_grade")]
#y_val <- class.ind(y_val) 
x_test <- test[,!(names(test)%in%"damage_grade")]
y_test <- test[,(names(test)%in%"damage_grade")]
y_test <- class.ind(y_test)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [3]:
library(keras)

In [4]:
x_train <- as.matrix(x_train)
y_train <- as.matrix(y_train)
x_val <- as.matrix(x_val)

In [5]:
params_ann <- function(nh,epochs){
    model <- keras_model_sequential() 
    model %>% 
      layer_dense(units = nh, input_shape = c(69)) %>% 
      layer_activation('relu') %>% 
      layer_dense(units = 3) %>% 
      layer_activation('softmax')
    
    model %>% compile(
      loss = "categorical_crossentropy",
      optimizer = optimizer_rmsprop(),
      metrics = c("accuracy")
    )
    
    history <- model %>% fit(
      x_train, y_train, verbose=0,
      epochs = epochs, batch_size = 4096, 
    )
    
    preds <- model %>% predict_classes(x_val)
    val_cm <- table(y_val, preds)
    
    return(perf_eval_multi(val_cm))
}

In [6]:
nh <- c(30,40,50)
epochs <- c(50,75,100)
ann_pre_search_result = matrix(0,length(nh)*length(epochs),4)
colnames(ann_pre_search_result) <- c("nh", "epochs","ACC", "BCR")

In [7]:
iter_cnt = 1
for ( i in 1:length(nh)){
    for ( j in 1:length(epochs)){
        ann_pre_search_result[iter_cnt,3:4] = params_ann(nh[i],epochs[j])
        ann_pre_search_result[iter_cnt,1] = nh[i]
        ann_pre_search_result[iter_cnt,2] = epochs[j]
        iter_cnt = iter_cnt+1
    }
}

In [8]:
ann_pre_search_result

nh,epochs,ACC,BCR
30,50,0.63294,0.4904217
30,75,0.63758,0.4960595
30,100,0.63480,0.4832430
40,50,0.63468,0.4774988
40,75,0.63876,0.4954295
40,100,0.63880,0.4968819
50,50,0.63586,0.5062350
50,75,0.63932,0.4964878
50,100,0.64090,0.4872817


In [9]:
set.seed(42)
trn_idx <- sample(1:nrow(data), 150000)
train <- data[trn_idx,]
tmp <- data[-trn_idx,]
val_idx <- sample(1:nrow(tmp),50000)
val <- tmp[val_idx,]
test <- tmp[-val_idx,]
x_train <- train[,!(names(train)%in%"damage_grade")]
y_train <- train[,(names(train)%in%"damage_grade")]
y_train <- class.ind(y_train)
x_val <- val[,!(names(val)%in%"damage_grade")]
y_val <- val[,(names(val)%in%"damage_grade")]
y_val <- class.ind(y_val) 
x_test <- test[,!(names(test)%in%"damage_grade")]
y_test <- test[,(names(test)%in%"damage_grade")]
#y_test <- class.ind(y_test)
all_x_train <- rbind(x_train,x_val)
all_y_train <- rbind(y_train,y_val)
all_x_train <- as.matrix(all_x_train)
all_y_train <- as.matrix(all_y_train)
x_test <- as.matrix(x_test)

In [10]:
repeat_ann <- function(nh,epochs){
    model <- keras_model_sequential() 
    model %>% 
      layer_dense(units = nh, input_shape = c(69)) %>% 
      layer_activation('relu') %>% 
      layer_dense(units = 3) %>% 
      layer_activation('softmax')
    
    model %>% compile(
      loss = "categorical_crossentropy",
      optimizer = optimizer_rmsprop(),
      metrics = c("accuracy")
    )
    
    history <- model %>% fit(
      all_x_train, all_y_train, verbose=0,
      epochs = epochs, batch_size = 4096, 
    )
    
    preds <- model %>% predict_classes(x_test)
    test_cm <- table(y_test, preds)
    
    return(perf_eval_multi(test_cm))
}

In [11]:
ann_result = matrix(0,30,2)
colnames(ann_result) <- c("ACC", "BCR")

In [12]:
iter_cnt <- 1
for ( i in 1:30){
    ann_result[iter_cnt,1:2] = repeat_ann(50,50)
    iter_cnt <- iter_cnt+1
}

In [13]:
ann_result

ACC,BCR
0.6397914,0.5034674
0.6453524,0.4930092
0.6414251,0.4978585
0.6414086,0.5092474
0.6396924,0.4920083
0.6387353,0.4943356
0.6400884,0.5127305
0.6405670,0.5067576
0.6408145,0.4819986
0.6390984,0.5224842


In [14]:
mean(ann_result[,1])

[1] 0.6402342

In [15]:
mean(ann_result[,2])

[1] 0.5044413

In [16]:
sd(ann_result[,1])

[1] 0.001757608

In [17]:
sd(ann_result[,2])

[1] 0.01142361